In [2]:
# 1. Imports
import pandas as pd
import numpy as np
import shap
import joblib
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# 2. Load FD001 CMAPSS test set (final-cycle per unit)
column_names = ['unit', 'cycle', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + \
               [f'sensor_measurement_{i}' for i in range(1, 22)]

df = pd.read_csv('C:/Users/ammar/SHAP_ML/datasets/train_FD001.txt', sep='\s+', header=None, names=column_names)

# Calculate RUL
rul = df.groupby('unit')['cycle'].max().reset_index()
rul.columns = ['unit', 'max_cycle']
df = df.merge(rul, on='unit')
df['RUL'] = df['max_cycle'] - df['cycle']
df.drop(columns=['max_cycle'], inplace=True)

# Drop uninformative features (as used before)
drop_cols = ['operational_setting_3', 'sensor_measurement_1', 'sensor_measurement_5',
             'sensor_measurement_10', 'sensor_measurement_16', 'sensor_measurement_18',
             'sensor_measurement_19']

X = df.drop(columns=['unit', 'cycle', 'RUL'] + drop_cols)
y = df['RUL']
feature_names = X.columns
n_features = X.shape[1]

# === 3. SHAP averaging setup ===
N_TRIALS = 25
shap_xgb_matrix = np.zeros((N_TRIALS, n_features))
shap_rf_matrix = np.zeros((N_TRIALS, n_features))

# Create output directory if needed
os.makedirs("C:/Users/ammar/SHAP_ML/trials_v/outputs", exist_ok=True)

print("Running 25 trials on CMAPSS (FD001)...")

# === 4. Run trials ===
for i in range(N_TRIALS):
    print(f"Trial {i+1}/{N_TRIALS}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100 + i)

    # XGBoost
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(X_train, y_train)
    explainer_xgb = shap.Explainer(xgb_model, X_train)
    shap_vals_xgb = explainer_xgb(X_test)
    shap_xgb_matrix[i, :] = np.abs(shap_vals_xgb.values).mean(axis=0)

    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=100 + i)
    rf_model.fit(X_train, y_train)
    explainer_rf = shap.Explainer(rf_model, X_train)
    shap_vals_rf = explainer_rf(X_test, check_additivity=False)
    shap_rf_matrix[i, :] = np.abs(shap_vals_rf.values).mean(axis=0)

# === 5. Average and save ===
xgb_mean = shap_xgb_matrix.mean(axis=0)
rf_mean = shap_rf_matrix.mean(axis=0)

shap_xgb_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': xgb_mean})
shap_rf_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': rf_mean})

shap_xgb_df.sort_values(by='mean_abs_shap', ascending=False).to_csv(
    'C:/Users/ammar/SHAP_ML/trials_v/outputs/shap_avg_cmaps_xgb.csv', index=False)
shap_rf_df.sort_values(by='mean_abs_shap', ascending=False).to_csv(
    'C:/Users/ammar/SHAP_ML/trials_v/outputs/shap_avg_cmaps_rf.csv', index=False)

# === 6. Plot Top 10 Features ===
def plot_top10(df, model_name):
    top10 = df.sort_values(by='mean_abs_shap', ascending=False).head(10)
    plt.figure(figsize=(10, 5))
    plt.barh(top10['feature'][::-1], top10['mean_abs_shap'][::-1])
    plt.xlabel('Mean |SHAP value|')
    plt.title(f'CMAPSS (FD001) - Top 10 Features ({model_name})')
    plt.tight_layout()
    plt.savefig(f"C:/Users/ammar/SHAP_ML/trials_v/top10_cmaps_{model_name.lower()}.png", dpi=300)
    plt.close()

plot_top10(shap_xgb_df, "XGBoost")
plot_top10(shap_rf_df, "RandomForest")

print("\nCMAPSS SHAP averaging complete. Results saved.")

Running 25 trials on CMAPSS (FD001)...
Trial 1/25


100%|===================| 4121/4127 [07:27<00:00]        

Trial 2/25


100%|===================| 4120/4127 [07:16<00:00]        

Trial 3/25


...

Trial 24/25


100%|===================| 4126/4127 [07:15<00:00]        

Trial 25/25


100%|===================| 4126/4127 [07:38<00:00]        


CMAPSS SHAP averaging complete. Results saved.
